In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
import seaborn as sns
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import FloatSlider, IntSlider, Dropdown, Button, VBox, HBox

import warnings
warnings.filterwarnings('ignore')

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
from qiskit.quantum_info import Statevector, partial_trace, entropy

import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
class QuantumAISimulator:
    def __init__(self, n_qubits=3):
        self.n_qubits = n_qubits
        self.backend = AerSimulator()
        self.pennylane_device = qml.device('default.qubit', wires=n_qubits)

    def create_circuit(self, gates_config):
        qc = QuantumCircuit(self.n_qubits)
        for gate in gates_config:
            gate_type = gate['type']
            qubit = gate['qubit']
            if gate_type == 'H': qc.h(qubit)
            elif gate_type == 'X': qc.x(qubit)
            elif gate_type == 'Y': qc.y(qubit)
            elif gate_type == 'Z': qc.z(qubit)
            elif gate_type == 'RX' and 'angle' in gate: qc.rx(gate['angle'], qubit)
            elif gate_type == 'RY' and 'angle' in gate: qc.ry(gate['angle'], qubit)
            elif gate_type == 'RZ' and 'angle' in gate: qc.rz(gate['angle'], qubit)
            elif gate_type == 'CNOT' and 'target' in gate: qc.cx(qubit, gate['target'])
        return qc

    def execute_circuit(self, circuit, shots=1024):
        compiled_circuit = transpile(circuit, self.backend)
        job = self.backend.run(compiled_circuit, shots=shots)
        result = job.result()
        statevector = result.get_statevector(circuit)
        measured_circuit = circuit.copy()
        measured_circuit.measure_all()
        qasm_backend = AerSimulator()
        qasm_compiled_circuit = transpile(measured_circuit, qasm_backend)
        qasm_job = qasm_backend.run(qasm_compiled_circuit, shots=shots)
        counts = qasm_job.result().get_counts(measured_circuit)
        return {'statevector': statevector, 'counts': counts, 'circuit': circuit}

    def calculate_entanglement_entropy(self, statevector):
        try:
            return min(entropy(statevector, [0]), 1.0)
        except:
            return 0.0

    def visualize_quantum_state(self, result, show_bloch=True, show_histogram=True):
        fig = plt.figure(figsize=(15, 10))
        # Circuit diagram
        ax1 = plt.subplot(3, 3, (1, 3))
        try:
            circuit_img = circuit_drawer(result['circuit'], output='mpl', style={'backgroundcolor': 'white'})
            ax1.imshow(circuit_img)
            ax1.axis('off')
        except:
            ax1.text(0.5, 0.5, 'Circuit Visualization Unavailable', ha='center', va='center', transform=ax1.transAxes)
        # Statevector
        ax2 = plt.subplot(3, 3, 4)
        statevector = result['statevector']
        amplitudes = np.abs(statevector.data)**2
        states = [f'|{i:0{self.n_qubits}b}⟩' for i in range(len(amplitudes))]
        bars = ax2.bar(range(len(amplitudes)), amplitudes, color=plt.cm.viridis(amplitudes / max(amplitudes)))
        ax2.set_xlabel('States'); ax2.set_ylabel('Probability')
        ax2.set_xticks(range(len(states))); ax2.set_xticklabels(states, rotation=45)
        for bar, amp in zip(bars, amplitudes):
            if amp > 0.01: ax2.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.01, f'{amp:.3f}', ha='center')
        # Histogram
        if show_histogram:
            ax3 = plt.subplot(3, 3, 5)
            counts = result['counts']
            all_states = [f'{i:0{self.n_qubits}b}' for i in range(2**self.n_qubits)]
            count_values = [counts.get(state, 0) for state in all_states]
            bars = ax3.bar(range(len(all_states)), count_values, color=plt.cm.plasma(np.array(count_values) / (max(count_values) if max(count_values)>0 else 1)))
            ax3.set_xlabel('Measured States')
            ax3.set_ylabel('Counts')
            ax3.set_xticks(range(len(all_states)))
            ax3.set_xticklabels([f'|{state}⟩' for state in all_states], rotation=45)
        # Bloch sphere (for qubit 0)
        if show_bloch:
            ax4 = plt.subplot(3, 3, 6, projection='3d')
            try:
                rho = partial_trace(statevector, [i for i in range(1, self.n_qubits)])
                pauli_x = np.array([[0,1],[1,0]])
                pauli_y = np.array([[0,-1j],[1j,0]])
                pauli_z = np.array([[1,0],[0,-1]])
                x = np.real(np.trace(rho.data@pauli_x))
                y = np.real(np.trace(rho.data@pauli_y))
                z = np.real(np.trace(rho.data@pauli_z))
                u = np.linspace(0, 2*np.pi, 50)
                v = np.linspace(0, np.pi, 50)
                sphere_x = np.outer(np.cos(u), np.sin(v))
                sphere_y = np.outer(np.sin(u), np.sin(v))
                sphere_z = np.outer(np.ones(np.size(u)), np.cos(v))
                ax4.plot_surface(sphere_x, sphere_y, sphere_z, alpha=0.3, color='lightblue')
                ax4.quiver(0,0,0,x,y,z,color='red', linewidth=3, arrow_length_ratio=0.1)
                ax4.quiver(0,0,0,1.2,0,0,color='black', alpha=0.6)
                ax4.quiver(0,0,0,0,1.2,0,color='black', alpha=0.6)
                ax4.quiver(0,0,0,0,0,1.2,color='black', alpha=0.6)
                ax4.set_xlim([-1.2,1.2]); ax4.set_ylim([-1.2,1.2]); ax4.set_zlim([-1.2,1.2])
                ax4.set_title('Bloch Sphere (Qubit 0)')
            except:
                ax4.text(0.5,0.5,0.5,'Bloch sphere\nerror.',ha='center',va='center')
        # Quantum metrics
        ax5 = plt.subplot(3, 3, (7, 9))
        ax5.axis('off')
        entanglement = self.calculate_entanglement_entropy(statevector)
        fidelity = np.abs(statevector.data[0]) ** 2
        coherence = np.sum(np.abs(statevector.data) ** 4)
        ax5.text(0.1, 0.9, f"Entanglement: {entanglement:.4f}\nFidelity: {fidelity:.4f}\nCoherence: {coherence:.4f}", 
                 transform=ax5.transAxes, fontsize=12, verticalalignment='top', fontfamily='monospace')
        plt.tight_layout(); plt.show()
        return {'entanglement': entanglement, 'fidelity': fidelity, 'coherence': coherence}


In [ ]:
# --- Helper AI/NN and PennyLane Functions ---

class QuantumCircuitOptimizer(nn.Module):
    """
    Neural network to optimize quantum circuit parameters
    """
    def __init__(self, input_size=10, hidden_size=64, output_size=6):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Tanh()
        )

    def forward(self, x):
        return self.network(x) * np.pi  # Output between -π and π


def create_variational_circuit(params, n_qubits=3):
    """Create parameterized quantum circuit using PennyLane"""
    @qml.qnode(simulator.pennylane_device)
    def circuit(parameters):
        # Layer 1: Rotation gates
        for i in range(n_qubits):
            qml.RY(parameters[i], wires=i)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        for i in range(n_qubits):
            qml.RZ(parameters[i + n_qubits], wires=i)
        return qml.state()
    return circuit(params)


def ai_optimize_circuit(target_state, iterations=100):
    """Use AI (Neural Network) to optimize quantum circuit parameters."""
    print("🧠 Starting AI optimization...")
    optimizer_net = QuantumCircuitOptimizer()
    optimizer = optim.Adam(optimizer_net.parameters(), lr=0.01)
    if target_state == 'superposition':
        target = np.ones(2 ** simulator.n_qubits) / np.sqrt(2 ** simulator.n_qubits)
    elif target_state == 'bell':
        target = np.zeros(2 ** simulator.n_qubits)
        target[0] = 1 / np.sqrt(2)
        target[-1] = 1 / np.sqrt(2)
    else:
        target = np.zeros(2 ** simulator.n_qubits)
        target[0] = 1
    losses = []
    best_params = None
    best_fidelity = 0
    for i in range(iterations):
        circuit_features = torch.randn(1, 10)
        optimized_params = optimizer_net(circuit_features).squeeze()
        state = create_variational_circuit(optimized_params.detach().numpy())
        fidelity = np.abs(np.vdot(state, target)) ** 2
        loss = 1 - fidelity
        optimizer.zero_grad()
        loss_tensor = torch.tensor(loss, requires_grad=True)
        loss_tensor.backward()
        optimizer.step()
        losses.append(loss.item())
        if fidelity > best_fidelity:
            best_fidelity = fidelity
            best_params = optimized_params.detach().numpy()
        if (i + 1) % 20 == 0:
            print(f"Iteration {i+1}: Fidelity = {fidelity:.4f}, Loss = {loss:.4f}")
    print(f"🎯 Optimization complete! Best fidelity: {best_fidelity:.4f}")
    # Loss and fidelity plot
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(losses)
    plt.title('Optimization Loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss (1 - Fidelity)')
    plt.grid(True)
    plt.subplot(1, 2, 2)
    fidelities = [1 - loss for loss in losses]
    plt.plot(fidelities)
    plt.title('Target State Fidelity')
    plt.xlabel('Iteration')
    plt.ylabel('Fidelity')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    return best_params, best_fidelity


In [ ]:
class QuantumCircuitOptimizer(nn.Module):
    def __init__(self, input_size=10, hidden_size=64, output_size=6):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Tanh()
        )

    def forward(self, x):
        return self.network(x) * np.pi

def create_variational_circuit(params, n_qubits=3):
    @qml.qnode(simulator.pennylane_device)
    def circuit(parameters):
        for i in range(n_qubits): qml.RY(parameters[i], wires=i)
        for i in range(n_qubits - 1): qml.CNOT(wires=[i, i+1])
        for i in range(n_qubits): qml.RZ(parameters[i + n_qubits], wires=i)
        return qml.state()
    return circuit(params)

def ai_optimize_circuit(target_state, iterations=100):
    print("🧠 Starting AI optimization...")
    optimizer_net = QuantumCircuitOptimizer()
    optimizer = torch.optim.Adam(optimizer_net.parameters(), lr=0.01)
    if target_state == 'superposition':
        target = np.ones(2 ** simulator.n_qubits) / np.sqrt(2 ** simulator.n_qubits)
    elif target_state == 'bell':
        target = np.zeros(2 ** simulator.n_qubits); target[0] = 1/np.sqrt(2); target[-1] = 1/np.sqrt(2)
    else:
        target = np.zeros(2 ** simulator.n_qubits); target[0] = 1
    losses, best_fidelity, best_params = [], 0, None
    for i in range(iterations):
        circuit_features = torch.randn(1, 10)
        optimized_params = optimizer_net(circuit_features).squeeze()
        state = create_variational_circuit(optimized_params.detach().numpy())
        fidelity = np.abs(np.vdot(state, target)) ** 2
        loss = 1 - fidelity
        optimizer.zero_grad()
        loss_tensor = torch.tensor(loss, requires_grad=True); loss_tensor.backward(); optimizer.step()
        losses.append(loss.item())
        if fidelity > best_fidelity: best_fidelity, best_params = fidelity, optimized_params.detach().numpy()
        if (i + 1) % 20 == 0: print(f"Iteration {i+1}: Fidelity={fidelity:.4f}, Loss={loss:.4f}")
    print(f"🎯 Optimization complete! Best fidelity: {best_fidelity:.4f}")
    plt.figure(figsize=(10, 4))
    plt.plot(losses)
    plt.title('Optimization Loss')
    plt.xlabel('Iteration'); plt.ylabel('Loss (1 - Fidelity)')
    plt.grid(True)
    plt.show()
    return best_params, best_fidelity


In [ ]:
def interactive_circuit_builder():
    gates_config = []
    gate_type = Dropdown(options=['H', 'X', 'Y', 'Z', 'RX', 'RY', 'RZ', 'CNOT'], value='H', description='Gate:')
    qubit_selector = IntSlider(value=0, min=0, max=simulator.n_qubits-1, description='Qubit:')
    angle_slider = FloatSlider(value=np.pi/2, min=0, max=2*np.pi, step=0.1, description='Angle:')
    target_qubit = IntSlider(value=1, min=0, max=simulator.n_qubits-1, description='Target:')
    add_gate_btn = Button(description='Add Gate', button_style='success')
    clear_btn = Button(description='Clear Circuit', button_style='danger')
    run_btn = Button(description='Run Circuit', button_style='primary')
    optimize_btn = Button(description='AI Optimize', button_style='warning')
    output = widgets.Output()
    def add_gate(b):
        with output:
            gate_info = {'type': gate_type.value, 'qubit': qubit_selector.value}
            if gate_type.value in ['RX','RY','RZ']:
                gate_info['angle'] = angle_slider.value
            elif gate_type.value == 'CNOT':
                gate_info['target'] = target_qubit.value
            gates_config.append(gate_info)
            print(f"✅ Added {gate_type.value} gate to qubit {qubit_selector.value}")
    def clear_circuit(b):
        with output:
            gates_config.clear()
            clear_output()
            print("🗑️ Circuit cleared")
    def run_circuit(b):
        with output:
            clear_output()
            if not gates_config: print("⚠️ No gates in circuit."); return
            print("🚀 Running quantum circuit...")
            circuit = simulator.create_circuit(gates_config)
            result = simulator.execute_circuit(circuit)
            simulator.visualize_quantum_state(result)
    def ai_optimize(b):
        with output:
            clear_output()
            print("🧠 AI optimization starting...")
            best_params, fidelity = ai_optimize_circuit('superposition', iterations=50)
            gates_config.clear()
            for i in range(simulator.n_qubits):
                gates_config.append({'type': 'RY', 'qubit': i, 'angle': best_params[i]})
            for i in range(simulator.n_qubits-1):
                gates_config.append({'type': 'CNOT', 'qubit': i, 'target': i+1})
            for i in range(simulator.n_qubits):
                gates_config.append({'type': 'RZ', 'qubit': i, 'angle': best_params[i + simulator.n_qubits]})
            print(f"🎯 AI optimization complete! Fidelity: {fidelity:.4f}")
    add_gate_btn.on_click(add_gate)
    clear_btn.on_click(clear_circuit)
    run_btn.on_click(run_circuit)
    optimize_btn.on_click(ai_optimize)
    controls = VBox([
        HBox([gate_type, qubit_selector]),
        HBox([angle_slider, target_qubit]),
        HBox([add_gate_btn, clear_btn]),
        HBox([run_btn, optimize_btn]),
        output
    ])
    display(controls)



In [ ]:
simulator = QuantumAISimulator(n_qubits=3)
interactive_circuit_builder()